# 18.0 简介

## 核心知识点

- **贝叶斯定理**：是朴素贝叶斯的理论基础，用于在已知先验概率和似然概率的情况下，计算后验概率：
  $$
  P(A|B) = \frac{P(B|A)P(A)}{P(B)}
  $$

- **朴素贝叶斯公式**：基于“特征条件独立”的“朴素”假设，分类器的核心公式为：
  $$
  P(y \mid x_1, \dots, x_j) = \frac{P(x_1, \dots, x_j \mid y)P(y)}{P(x_1, \dots, x_j)}
  $$
  - $P(y \mid x_1, \dots, x_j)$：后验概率，即给定特征 $x_1, \dots, x_j$ 时，样本属于类别 $y$ 的概率。
  - $P(x_1, \dots, x_j \mid y)$：似然概率，即类别 $y$ 下，特征 $x_1, \dots, x_j$ 出现的概率。
  - $P(y)$：先验概率，即类别 $y$ 在数据集中出现的概率。
  - $P(x_1, \dots, x_j)$：边缘概率，所有特征的联合概率（计算时可忽略，因为它对所有类别恒定）。

- **核心优势**：
  1. 方法直观，易于理解。
  2. 对小样本数据也能有效工作。
  3. 训练和预测的计算成本低。
  4. 对参数设置不敏感，结果稳定。

- **两个重要假设**：
  1. 每个特征的似然概率 $P(x_j \mid y)$ 服从特定的统计分布（如正态、多项式、伯努利分布）。
  2. 所有特征之间相互独立（“朴素”假设，虽常不成立，但实际效果良好）。

# 18.1 为连续的数据训练分类器

问题描述：
数据特征为连续值，训练朴素贝叶斯分类器。

In [1]:
#加载库
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 创建高斯朴素贝叶斯对象
classifier = GaussianNB()

# 训练模型
model = classifier.fit(features, target)

In [2]:
# 创建一个观察值
new_observation = [[4,4,4,0.4]]

# 预测分类
model.predict(new_observation)

array([1])

In [3]:
# 给定每个分类的先验概率
clf = GaussianNB(priors=[0.25,0.25,0.5])

#训练模型
model = classifier.fit(features, target)

## 核心知识点

- **高斯朴素贝叶斯 (GaussianNB)**：假设每个类别 $(y)$ 下，特征 $(x_j)$ 的似然概率服从正态（高斯）分布：
  $$
  p(x_j \mid y) = \frac{1}{\sqrt{2\pi\sigma_y^2}} e^{-\frac{(x_j - \mu_y)^2}{2\sigma_y^2}}
  $$
其中 $\mu_y$ 和 $\sigma_y^2$ 是类别 $y$ 中特征 $x_j$ 的均值和方差

- **适用场景**：所有特征均为连续值的场景。

- **关键参数**：
  - `priors`：手动指定每个类别的先验概率。若不指定，模型会从数据中自动计算。

- **注意**：GaussianNB 输出的预测概率（`predict_proba`）未经校准，不可直接信任，需使用校准方法（见 18.4 节）。

# 18.2 为离散数据和计数数据训练分类器

问题描述：
数据特征为离散值或计数数据（如文本词频），训练朴素贝叶斯分类器。

In [4]:
# 加载库
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

# 创建文本
text_data = np.array(['I love Brazil. Brazil!',
                      'Brazil is best',
                      'Germary beats both'])

# 创建词袋
count = CountVectorizer()
bag_of_words = count.fit_transform(text_data)

# 创建特征矩阵
features = bag_of_words.toarray()

# 创建目标向量
target = np.array([0,0,1])

# 给定每个分类的先验概率，创建一个多项式朴素贝叶斯对象
classifier = MultinomialNB(class_prior = [0.25,0.5])

# 训练模型
model = classifier.fit(features, target)

In [5]:
#创建一个观察值
new_observation = [[0,0,0,1,0,1,0]]

# 预测新观察值的分类
model.predict(new_observation)

array([0])

## 核心知识点

- **多项式朴素贝叶斯 (MultinomialNB)**：假设特征服从多项式分布，适用于离散或计数数据。

- **适用场景**：文本分类（如词袋模型、TF-IDF）、评分数据（1-5分）等。

- **关键参数**：
  - `class_prior`：手动指定每个类别的先验概率。若不指定，从数据中学习。
  - `fit_prior`：设置为 `False` 时，使用均匀分布作为先验概率。
  - `alpha`：平滑参数，用于处理零概率问题，默认值为 1.0（拉普拉斯平滑）。

# 18.3 为具有二元特征的数据训练朴素贝叶斯分类器

问题描述：
数据特征为二元值（如 0/1、True/False），训练朴素贝叶斯分类器。

In [8]:
# 加载库
import numpy as np
from sklearn.naive_bayes import BernoulliNB

# 创建三个二元特征
features = np.random.randint(2,size = (100,3))

# 创建一个二元目标向量
target = np.random.randint(2,size = (100,1)).ravel()

# 给定每个分类的先验概率，创建一个多项式伯努利朴素贝叶斯对象
classifier = BernoulliNB(class_prior=[0.25,0.5])

#训练模型
model = classifier.fit(features, target)

In [9]:
# 使用均匀先验概率
model_uniform_prior = BernoulliNB(class_prior=None, fit_prior=False)

## 核心知识点

- **伯努利朴素贝叶斯 (BernoulliNB)**：假设所有特征均为二元分类（如 one-hot 编码后的名义特征）。

- **适用场景**：文本分类（表示某个词是否在文档中出现）、二元特征的分类任务。

- **关键参数**：
  - `class_prior`：手动指定每个类别的先验概率。
  - `fit_prior`：设置为 `False` 时，使用均匀分布作为先验概率。
  - `alpha`：平滑参数，与 `MultinomialNB` 作用相同。

# 18.4 校准预测概率

问题描述：
校准朴素贝叶斯分类器输出的预测概率，使其具有可解释性。

In [11]:
# 加载库
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import  CalibratedClassifierCV

# 加载数据
iris = datasets.load_iris()
features = iris.data
target = iris.target

# 创建高斯朴素贝叶斯对象
classifier = GaussianNB()

# 创建使用sigmoid校准调校过的交叉验证模型
classifier_sigmoid = CalibratedClassifierCV(classifier,cv = 2,method='sigmoid')

# 调校概率
classifier_sigmoid.fit(features, target)

# 创建新的观察值
new_observation = [[2.6,2.6,2.6,0.4]]

# 查看调校过的概率
classifier_sigmoid.predict_proba(new_observation)


array([[0.31859971, 0.63663451, 0.04476578]])

## 核心知识点

- **朴素贝叶斯的概率问题**：其输出的预测概率往往非常极端（趋近于0或1），与真实世界的概率不符，因此需要校准。

- **校准方法**：在 scikit-learn 中，使用 `CalibratedClassifierCV` 类，通过 k 折交叉验证来校准预测概率：
  - **Platt sigmoid 模型**：适用于小样本量（如150个样本），通过训练一个逻辑回归模型来校准。
  - **保序回归（Isotonic Regression）**：无参数方法，但在小样本下容易过拟合。

- **核心作用**：校准后的概率可以被解释为真实世界的置信度，而不仅仅是类别排序的依据。

# 第18章 朴素贝叶斯总结
1. **核心原理**
    基于**贝叶斯定理** + 极强的**特征条件独立**（朴素）假设，通过先验概率和似然概率计算后验概率，实现分类。
    分类核心公式：
   $
    P(y \mid x_1,...,x_n) \propto P(y)\prod_{i=1}^n P(x_i \mid y)
   $

2. **三类常用朴素贝叶斯（按特征类型选）**
    - **高斯朴素贝叶斯 GaussianNB**
      适用：**连续数值特征**，假设特征服从正态分布。
    - **多项式朴素贝叶斯 MultinomialNB**
      适用：**离散计数特征**（如文本词频、词袋），带拉普拉斯平滑 `alpha`。
    - **伯努利朴素贝叶斯 BernoulliNB**
      适用：**二元 0/1 特征**（如词是否出现）。

3. **关键参数**
    - `class_prior / priors`：手动指定类别先验概率
    - `alpha`：平滑参数，解决零概率问题
    - `fit_prior`：是否从数据中学习先验概率

4. **重要问题：概率校准**
    朴素贝叶斯输出的概率**偏极端、不可直接信**，需用
    `CalibratedClassifierCV` 结合 **Platt sigmoid** 或保序回归校准，让概率接近真实置信度。

5. **特点**
    - 训练/预测极快、对小样本友好、适合高维数据
    - 独立假设通常不成立，但实际分类效果依然很稳
    - 最大短板：原生概率不准，必须校准才能用于置信度判断